In [50]:
import pandas as pd
import numpy as np
import os
import datetime as dt

from statsmodels.tsa.ar_model import AR, ARResults

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [67]:
file_path = '{}/nem-data/nemde_clean/df_DUID_CO2_mean.pkl'.format(os.environ['HOME'])
df = pd.read_pickle(file_path)
df = df.sort_index()
df = df[df.index >= dt.datetime(2009,9,1,0,0,0)] #minimum time of interconncector

demand = pd.read_csv('{}/nem-data/demand.csv'.format(os.environ['HOME']), index_col=-1, parse_dates=True)
demand.drop(columns=["SETTLEMENTDATE", "I", "INTERVENTION"], inplace=True)
demand = demand[(demand.index >= df.index.min()) & (demand.index <= df.index.max())]

assert demand.index.min() == df.index.min()
assert demand.index.max() == df.index.max()

demand = demand.groupby(demand.index).mean()

df["demand"] = demand.TOTALDEMAND
df["demand_capacity"] = demand.TOTALDEMAND/demand.AVAILABLEGENERATION

interconnectors = pd.read_csv('{}/nem-data/interconnectors.csv'.format(os.environ['HOME']), index_col=-1, parse_dates=True)
interconnectors.drop(columns=["SETTLEMENTDATE", "I", "INTERCONNECTORID"], inplace=True)
interconnectors = interconnectors[(interconnectors.index >= df.index.min()) & (interconnectors.index <= df.index.max())]

assert interconnectors.index.min() == df.index.min()
assert interconnectors.index.max() == df.index.max()

interconnectors = interconnectors.groupby(interconnectors.index).mean()
df["interconnector"] = interconnectors.MWFLOW


df.head()

,Price,CO2E_EMISSIONS_FACTOR,demand,demand_capacity,interconnector
2009-09-01 00:00:00,21.64992,0.986067,1667.12,0.603199,302.86
2009-09-01 00:05:00,21.49293,0.976360,1657.52,0.599962,300.00
2009-09-01 00:10:00,21.48960,0.976889,1650.15,0.596877,290.52
2009-09-01 00:15:00,21.14641,1.032780,1630.66,0.589438,260.75
2009-09-01 00:20:00,20.94913,0.975655,1628.96,0.587282,256.98


In [68]:
df = df.loc[:,["CO2E_EMISSIONS_FACTOR"]]
df.head()

,CO2E_EMISSIONS_FACTOR
2009-09-01 00:00:00,0.986067
2009-09-01 00:05:00,0.976360
2009-09-01 00:10:00,0.976889
2009-09-01 00:15:00,1.032780
2009-09-01 00:20:00,0.975655


In [69]:
train = df[df.index < dt.datetime(2017,5,31,0,0,0)]
test = df[df.index >= dt.datetime(2017,5,31,0,0,0) + dt.timedelta(minutes=5) * 12]

In [70]:
print(train.index.max())
print(test.index.min())

2017-05-30 23:55:00
2017-05-31 01:00:00


In [71]:
model = AR(train.CO2E_EMISSIONS_FACTOR)

In [72]:
AR12fit = model.fit(maxlag=12)

MissingDataError: exog contains inf or nans

In [73]:
AR12fit.k_ar

12

In [65]:
pred = AR12fit.predict(start = 13, end = train.shape[0])

In [ ]:
import

In [42]:
df[df.index >= dt.datetime(2017,5,31,0,0,0) + dt.timedelta(minutes=5) * 12]["CO2E_EMISSIONS_FACTOR"]

2017-05-31 01:00:00    0.663890
2017-05-31 01:05:00    0.868261
2017-05-31 01:10:00    0.605537
2017-05-31 01:15:00    0.726337
2017-05-31 01:20:00    0.726337
                         ...   
2019-08-01 03:35:00    0.573436
2019-08-01 03:40:00    0.573436
2019-08-01 03:45:00    0.573436
2019-08-01 03:50:00    0.456409
2019-08-01 03:55:00    0.380340
Name: CO2E_EMISSIONS_FACTOR, Length: 208949, dtype: float64

In [66]:
train.plot()
pred.plot()

plt.show()

ValueError: year 0 is out of range

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fa95d4d3b70> (for post_execute):


ValueError: year 0 is out of range

ValueError: year 0 is out of range